In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy
import pickle
from glob import glob
from typing import Any, Dict, List, Tuple, Union
import pandas as pd

"""Change to the data folder"""
new_path = "./new_train/new_train"
test_path = './new_val_in/new_val_in'
subset_test_path = './new_train/train_subset'
# number of sequences in each dataset
# train:205942  val:3200 test: 36272 
# sequences sampled at 10HZ rate

### Create a dataset class 

In [3]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
        if self.transform:
            data = self.transform(data)

        return data


# intialize a dataset
val_dataset  = ArgoverseDataset(data_path=new_path)
test_dataset = ArgoverseDataset(data_path=test_path)

### Create a loader to enable batch processing

In [93]:
batch_sz = 100

def train_agents_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = numpy.concatenate([numpy.dstack([scene['p_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch])
    out = numpy.concatenate([numpy.dstack([scene['p_out'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch])
    inp = torch.Tensor(inp)
    out = torch.Tensor(out)
    return [inp, out]

def train_all_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = numpy.concatenate([numpy.dstack([scene['p_in'][['dummy' not in word for word in scene['track_id'][:,0,0]],:,:]]) for scene in batch])
    out = numpy.concatenate([numpy.dstack([scene['p_out'][['dummy' not in word for word in scene['track_id'][:,0,0]],:,:]]) for scene in batch])
    inp = torch.Tensor(inp)
    out = torch.Tensor(out)
    return [inp, out]
    
def test_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = numpy.concatenate([numpy.dstack([scene['p_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch])
    inp = torch.Tensor(inp)
    idx = [numpy.dstack([scene['scene_idx']]) for scene in batch]
    return inp, idx
    
train_agent_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = True, collate_fn=train_agents_collate, num_workers=0)

train_all_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = True, collate_fn=train_all_collate, num_workers=0)

test_loader = DataLoader(test_dataset,batch_size=batch_sz, shuffle = True, collate_fn=test_collate, num_workers=0)

In [81]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class EncoderRNN(nn.Module):
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    
    def __init__(self,
                 input_size = 2,
                 embedding_size = 8,
                 hidden_size = 16):
        
        super(EncoderRNN, self).__init__()
        
        self.hidden_size = hidden_size
        self.linear = nn.Linear(input_size, embedding_size)
        self.lstm = nn.LSTMCell(embedding_size, hidden_size)

    def forward(self, x, hidden):
        embedded = F.relu(self.linear(x))
        hidden = self.lstm(embedded, hidden)
        return hidden


class DecoderRNN(nn.Module):
    """Decoder Network."""
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    def __init__(self, embedding_size=8, hidden_size=16, output_size=2):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.linear1 = nn.Linear(output_size, embedding_size)
        self.lstm = nn.LSTMCell(embedding_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        embedded = F.relu(self.linear1(x))
        hidden = self.lstm(embedded, hidden)
        output = self.linear2(hidden[0])
        return output, hidden


In [91]:
from tqdm import tqdm_notebook as tqdm

def train(encoder, decoder, device, train_loader, encoder_optimizer, decoder_optimizer, epoch, log_interval=10000):    
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    
    iterator = tqdm(train_loader, total=int(len(train_loader)))
    counter = 0
    criterion = nn.MSELoss()
    
    for i_batch, sample_batch in enumerate(train_loader):
        
        inp, out = sample_batch
#         print(inp.shape)
        # preprocessing more ????
#         inp = inp[:,0,:,:]
#         out = out[:,0,:,:]
        
        #inp - inp[0] for all in whaetver
        x_offset = []
        y_offset = []
        for i in range(inp.shape[0]):
            x_offset.append(inp[i][0][0].detach().clone())
            y_offset.append(inp[i][0][1].detach().clone())
    
        for j in range(inp.shape[0]):
            for i in range(inp.shape[1]):
                inp[j][i][0] = inp[j][i][0] - x_offset[j]
                inp[j][i][1] = inp[j][i][1] - y_offset[j]

        #outoput whatever
        for j in range(out.shape[0]):
            for i in range(out.shape[1]):
                out[j][i][0] = out[j][i][0] - x_offset[j]
                out[j][i][1] = out[j][i][1] - y_offset[j]
        
        _input, target = inp.to(device), out.to(device)
        
        encoder.train()
        decoder.train()
        
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
        
        
        #encoder 
        batch_size = _input.shape[0]
        input_length = _input.shape[1]
        output_length = target.shape[1]
        feature_len = _input.shape[2]
        input_shape = _input.shape[2]
        
        encoder_hidden = (torch.zeros(batch_size, encoder.module.hidden_size).to(device), 
                          torch.zeros(batch_size, encoder.module.hidden_size).to(device))
        
        loss = 0
        
        # Encode observed trajectory
        for ei in range(input_length):
            encoder_input = _input[:, ei, :]
            encoder_hidden = encoder(encoder_input, encoder_hidden)

        # Initialize decoder input with last coordinate in encoder
        decoder_input = encoder_input[:, :2]

        # Initialize decoder hidden state as encoder hidden state
        decoder_hidden = encoder_hidden

        decoder_outputs = torch.zeros(target.shape).to(device)

        # Decode hidden state in future trajectory
        for di in range(30):
            decoder_output, decoder_hidden = decoder(decoder_input,
                                                     decoder_hidden)
            decoder_outputs[:, di, :] = decoder_output

            # Update loss
            loss += torch.sqrt(criterion(decoder_output[:, :2], target[:, di, :2]))

            # Use own predictions as inputs at next step
            decoder_input = decoder_output

        # Get average loss for pred_len
        loss = loss / 30

        # Backpropagate
        loss.backward()
        encoder_optimizer.step()
        decoder_optimizer.step()
        
        file1 = open("loss_steps.txt", "a")  # append mode
        file1.write(str(loss.item()) + ",")
        file1.close()
        
#       output = model(data)
#         loss = MSELoss(output, target)
        counter += 1
        iterator.set_postfix(loss=(loss.item()))
    

In [94]:
device = "cuda"
encoder = EncoderRNN(input_size=2)
decoder = DecoderRNN(output_size=2)

encoder = nn.DataParallel(encoder)
decoder = nn.DataParallel(decoder)

encoder.to(device)
decoder.to(device)

encoder_optimizer = torch.optim.Adam(encoder.parameters())
decoder_optimizer = torch.optim.Adam(decoder.parameters())

num_epoch = 1

for epoch in range(1, num_epoch + 1):
    train(encoder, decoder, device, train_all_loader, encoder_optimizer, decoder_optimizer, epoch)
#         train(encoder, decoder, device, train_agent_loader, encoder_optimizer, decoder_optimizer, epoch)
#         predict(model, device, test_loader)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [95]:
def infer_absolute(
        test_loader: torch.utils.data.DataLoader,
        encoder: EncoderRNN,
        decoder: DecoderRNN,
#         start_idx: int,
#         forecasted_save_dir: str,
#         model_utils: ModelUtils,
):
    """Infer function for non-map LSTM baselines and save the forecasted trajectories.
    
    referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting
    
    Args:
        test_loader: DataLoader for the test set
        encoder: Encoder network instance
        decoder: Decoder network instance
        start_idx: start index for the current joblib batch
        forecasted_save_dir: Directory where forecasted trajectories are to be saved
        model_utils: ModelUtils instance

    """
    
    forecasted_trajectories = {}

    for i, (_input, idx) in enumerate(test_loader):
        
#         _input = _input[:,0,:,:]
        
        #inp - inp[0] for all in whaetver
        x_offset = []
        y_offset = []
        for i in range(_input.shape[0]):
            x_offset.append(_input[i][0][0].detach().clone())
            y_offset.append(_input[i][0][1].detach().clone())
    
        for j in range(_input.shape[0]):
            for i in range(_input.shape[1]):
                _input[j][i][0] = _input[j][i][0] - x_offset[j]
                _input[j][i][1] = _input[j][i][1] - y_offset[j]

        _input = _input.to(device)

        # Set to eval mode
        encoder.eval()
        decoder.eval()

        # Encoder
        batch_size = _input.shape[0]
        input_length = _input.shape[1]
        input_shape = _input.shape[2]

        # Initialize encoder hidden state
        encoder_hidden = (torch.zeros(batch_size, encoder.module.hidden_size).to(device), 
                          torch.zeros(batch_size, encoder.module.hidden_size).to(device))
       
        # Encode observed trajectory
        for ei in range(input_length):
            encoder_input = _input[:, ei, :]
            encoder_hidden = encoder(encoder_input, encoder_hidden)

        # Initialize decoder input with last coordinate in encoder
        decoder_input = encoder_input[:, :2]

        # Initialize decoder hidden state as encoder hidden state
        decoder_hidden = encoder_hidden

        decoder_outputs = torch.zeros(
            (batch_size, 30, 2)).to(device)

        # Decode hidden state in future trajectory
        for di in range(30):
            decoder_output, decoder_hidden = decoder(decoder_input,
                                                     decoder_hidden)
            decoder_outputs[:, di, :] = decoder_output

            # Use own predictions as inputs at next step
            decoder_input = decoder_output

        for i in range(30):
            for j in range(batch_size):
                decoder_outputs[j,i,0] = decoder_outputs[j,i,0] + x_offset[j]
                decoder_outputs[j,i,1] = decoder_outputs[j,i,1] + y_offset[j]
            
                if (idx[j][0][0][0] in forecasted_trajectories):
                    forecasted_trajectories[idx[j][0][0][0]].append(decoder_outputs[j,i,:].tolist())
                else:
                    forecasted_trajectories[idx[j][0][0][0]] = [decoder_outputs[j,i,:].tolist()]
                
    return(forecasted_trajectories)

In [96]:
output = infer_absolute(test_loader, encoder, decoder)

In [97]:
import pandas as pd
df = pd.DataFrame.from_dict(output, orient='index')
df

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
9016,"[234.31005859375, 1363.3331298828125]","[233.4396209716797, 1361.9183349609375]","[233.26112365722656, 1361.655517578125]","[233.138427734375, 1361.5330810546875]","[232.91961669921875, 1361.3482666015625]","[232.47251892089844, 1360.9827880859375]","[231.7026824951172, 1360.358154296875]","[230.7525177001953, 1359.5892333984375]","[229.95875549316406, 1358.9473876953125]","[229.47557067871094, 1358.5567626953125]",...,"[229.0413818359375, 1358.2030029296875]","[229.04232788085938, 1358.2034912109375]","[229.04327392578125, 1358.2041015625]","[229.04425048828125, 1358.20458984375]","[229.0452117919922, 1358.205078125]","[229.0461883544922, 1358.20556640625]","[229.04714965820312, 1358.2060546875]","[229.04812622070312, 1358.20654296875]","[229.04910278320312, 1358.20703125]","[229.05007934570312, 1358.2076416015625]"
12770,"[1823.99951171875, 430.30401611328125]","[1824.6781005859375, 431.106689453125]","[1824.9744873046875, 431.6295471191406]","[1825.1258544921875, 432.14044189453125]","[1825.24072265625, 432.7698669433594]","[1825.3531494140625, 433.53033447265625]","[1825.4608154296875, 434.3126525878906]","[1825.54638671875, 434.9490051269531]","[1825.6007080078125, 435.3540954589844]","[1825.6295166015625, 435.56658935546875]",...,"[1825.6553955078125, 435.7427978515625]","[1825.6553955078125, 435.7428283691406]","[1825.6553955078125, 435.74285888671875]","[1825.655517578125, 435.7428894042969]","[1825.655517578125, 435.7428894042969]","[1825.655517578125, 435.7428894042969]","[1825.655517578125, 435.742919921875]","[1825.655517578125, 435.742919921875]","[1825.655517578125, 435.742919921875]","[1825.655517578125, 435.742919921875]"
34897,"[1812.036865234375, 430.17877197265625]","[1810.2088623046875, 428.22845458984375]","[1808.7978515625, 427.1519470214844]","[1807.6353759765625, 426.3677062988281]","[1806.8240966796875, 425.8850402832031]","[1806.3038330078125, 425.517578125]","[1805.9368896484375, 424.9245910644531]","[1805.6182861328125, 423.8333740234375]","[1805.34375, 422.5434265136719]","[1805.175048828125, 421.72808837890625]",...,"[1805.65625, 418.00836181640625]","[1805.788818359375, 417.31561279296875]","[1805.8603515625, 416.9420166015625]","[1805.888916015625, 416.792724609375]","[1805.898681640625, 416.7416076660156]","[1805.90185546875, 416.72509765625]","[1805.90283203125, 416.71990966796875]","[1805.9031982421875, 416.7182922363281]","[1805.9031982421875, 416.7177734375]","[1805.9033203125, 416.7176208496094]"
34978,"[702.3152465820312, 1145.075927734375]","[701.1241455078125, 1141.653076171875]","[700.4012451171875, 1138.6319580078125]","[700.0833129882812, 1136.9151611328125]","[699.741455078125, 1136.0157470703125]","[699.2310791015625, 1135.2945556640625]","[698.6336669921875, 1134.6197509765625]","[698.096923828125, 1134.06005859375]","[697.7144775390625, 1133.670654296875]","[697.4865112304688, 1133.4366455078125]",...,"[697.2734375, 1133.1754150390625]","[697.2750244140625, 1133.1756591796875]","[697.276611328125, 1133.1759033203125]","[697.2780151367188, 1133.17626953125]","[697.2794189453125, 1133.1766357421875]","[697.28076171875, 1133.177001953125]","[697.2820434570312, 1133.1773681640625]","[697.2833251953125, 1133.1778564453125]","[697.2845458984375, 1133.17822265625]","[697.2857666015625, 1133.1787109375]"
15978,"[1666.2255859375, 288.8434143066406]","[1667.19091796875, 289.3826599121094]","[1667.4456787109375, 289.5143127441406]","[1667.9674072265625, 289.9256591796875]","[1669.2099609375, 290.9508361816406]","[1670.527587890625, 292.0140075683594]","[1671.1322021484375, 292.4914855957031]","[1671.317138671875, 292.6434631347656]","[1671.3876953125, 292.71484375]","[1671.443359375, 292.7869873046875]",...,"[1674.13232421875, 296.9613952636719]","[1674.1591796875, 297.01422119140625]","[1674.1722412109375, 297.0489807128906]","[1674.1790771484375, 297.0778503417969]","[1674.1832275390625, 297.10736083984375]","[1674.1866455078125

In [98]:
for i in range(30):
    df[['v{}'.format((i*2)+1), 'v{}'.format((i*2)+2)]] = pd.DataFrame(df.get(i).tolist(), index=df.index)

In [99]:
import numpy as np
dropped_cols = list(np.arange(30))
df2 = df.drop(dropped_cols, axis=1)
df2.index.name = 'ID'
df2

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,...,v51,v52,v53,v54,v55,v56,v57,v58,v59,v60
ID,,,,,,,,,,,,,,,,,,,,,
9016,234.310059,1363.333130,233.439621,1361.918335,233.261124,1361.655518,233.138428,1361.533081,232.919617,1361.348267,...,229.046188,1358.205566,229.047150,1358.206055,229.048126,1358.206543,229.049103,1358.207031,229.050079,1358.207642
12770,1823.999512,430.304016,1824.678101,431.106689,1824.974487,431.629547,1825.125854,432.140442,1825.240723,432.769867,...,1825.655518,435.742889,1825.655518,435.742920,1825.655518,435.742920,1825.655518,435.742920,1825.655518,435.742920
34897,1812.036865,430.178772,1810.208862,428.228455,1808.797852,427.151947,1807.635376,426.367706,1806.824097,425.885040,...,1805.901855,416.725098,1805.902832,416.719910,1805.903198,416.718292,1805.903198,416.717773,1805.903320,416.717621
34978,702.315247,1145.075928,701.124146,1141.653076,700.401245,1138.631958,700.083313,1136.915161,699.741455,1136.015747,...,697.280762,1133.177002,697.282043,1133.177368,697.283325,1133.177856,697.284546,1133.178223,697.285767,1133.178711
15978,1666.225586,288.843414,1667.190918,289.382660,1667.445679,289.514313,1667.967407,289.925659,1669.209961,290.950836,...,1674.186646,297.141357,1674.190308,297.182526,1674.194702,297.233276,1674.200317,297.295990,1674.207764,297.373260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29848,2152.728516,815.541870,2151.616699,814.383240,2151.203857,814.169373,2151.011719,814.268433,2150.861816,814.334351,...,2147.127197,812.535034,2146.938965,812.532959,2146.725830,812.490295,2146.466064,812.377319,2146.121094,812.145508
13067,746.648499,1280.489014,746.611816,1282.011475,746.597351,1283.115601,746.491516,1283.479248,746.327393,1283.608154,...,747.192932,1291.570557,747.385071,1291.829102,747.571777,1292.080078,747.754578,1292.325195,747.934692,1292.565186
36574,474.189941,1144.470093,472.960907,1144.359131,472.072998,1144.329224,471.368958,1144.368774,470.807434,1144.477173,...,468.648438,1144.023560,468.636810,1144.014160,468.627472,1144.007080,468.620209,1144.002197,468.614868,1143.999268


In [100]:
df2.to_csv("outputs1ep_train.csv", index=True)